In [1]:
import pandas as pd 
import numpy as np 
import csv 
from Bio import SeqIO
from itertools import islice 


In [18]:
#Reads in Fasta reference file and stores information in a list using SeqIO
#Use information about the length of first and second chromosome to determine early and late mutations 

def referenceStats(pathToReferenceFasta):
    
    #Creates a list of records for each sequence in the fasta file 
    records = list(SeqIO.parse(pathToReferenceFasta, "fasta"))
    
    #Loop through the record objects and yield the chromosome name and length of the chromosome 
    for num in range(len(records)):
        sample_len = (len(records[num].seq))
        yield (records[num].name, sample_len)
    
# Test to see if function works 
genObj = referenceStats("/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/rradio_copy.fa")
for item in genObj: 
    print(item)


('gi|221721649|gb|CP000628.1|', 4005130)
('gi|221725460|gb|CP000629.1|', 2650913)
('gi|221727862|gb|CP000630.1|', 184668)
('gi|221728079|gb|CP000631.1|', 388169)
('gi|221728482|gb|CP000632.1|', 44420)


In [21]:
#List of origins and termini according to Cd Viz for each organism 
rradioOrigin = 496696 
rradioTerm = 2499261
vitisOrigin = 336131 
vitisTerm = 2199319 
tumOrigin = 2765000
tumTerm = 1344210

#Function to adjust postion according to origin and terminus in Cd Viz 
#Shifts the origin given by Cd Viz to 1 and adjusts other positions accordingly 
def adjustPosition(pathToReferenceFasta, position, origin):
    
    #If position is less than the origin, subtract from the full length of the chromosome 
    if position < origin: 
        genObj = referenceStats(pathToReferenceFasta)
        for i in islice(genObj, 0, 1):
            fullchromLength = i[1]  
        adjPosition = fullchromLength - (origin -(position + 1))
        
    #if position is greater than the origin, subtract origin from position and add 1 since the origin is at position 1 
    elif position > origin: 
        adjPosition = (position - origin) + 1  
        
    #Readjust origin to position 1 
    elif position == origin: 
        adjPosition = 1 
    
    else: 
        adjPosition = "No position"
    
    return adjPosition 


#Test
# print(adjustPosition("/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/rradio_copy.fa",496695, rradioOrigin))

4005129


In [23]:
#Calculate Transitions and transversions
#Goes through an excel file with multiple samples of VCF calls
#Note on formatting of excel file: keeps columns of the VCF, but must have a Sample Name column to distinguish the sample
#Generates new dictionary for each sample with positions of each variant call serving as keys
#Values are the sample name and mutation type
#Adjusts position according to origin and terminus in Cd Viz 

def transitionsTransversions(pathToExcelFile, pathToReferenceFasta, origin):
    
    #Reads in all sheets in an excel file and stores as a dictionary 
    excelTotalSheets = pd.read_excel(pathToExcelFile, sheet_name=None)
    
    #Get keys in the dictionary 
    keysList = excelTotalSheets.keys()
    
    #Loop through each sample in the excel file 
    for sample in keysList: 
        count = 0 
        value = pd.DataFrame(excelTotalSheets[str(sample)])
        sampleDict = {}
        while count < len(value.index):
            sampleName = value.iloc[count]["Sample_name"]
            sampleDict["Sample_name"] = sampleName
            position = value.iloc[count]["POS"]
            adjPos = adjustPosition(pathToReferenceFasta, position, origin)
            ref = value.iloc[count]["REF"]
            alt = value.iloc[count]["ALT"]
            
            if ref == "A" and alt == "G": 
                sampleDict[adjPos] = "AT_GC"
                count += 1 
            elif ref == "T" and alt == "C":
                sampleDict[adjPos] = "AT_GC"
                count += 1
            elif ref == "G" and alt == "A":
                sampleDict[adjPos] = "GC_AT"
                count += 1
            elif ref == "C" and alt == "T":
                sampleDict[adjPos] = "GC_AT"
                count += 1
            elif ref == "A" and alt == "T": 
                sampleDict[adjPos] = "AT_TA"
                count += 1
            elif ref == "T" and alt == "A": 
                sampleDict[adjPos] = "AT_TA"
                count += 1
            elif ref == "G" and alt == "T":
                sampleDict[adjPos] = "GC_TA"
                count += 1
            elif ref == "C" and alt == "A":
                sampleDict[adjPos] = "GC_TA"
                count += 1
            elif ref == "A" and alt == "C":
                sampleDict[adjPos] = "AT_CG"
                count += 1
            elif ref == "T" and alt == "G":
                sampleDict[adjPos] = "AT_CG"
                count += 1
            elif ref == "G" and alt == "C":
                sampleDict[adjPos] = "GC_CG"
                count += 1
            elif ref == "C" and alt == "G":
                sampleDict[adjPos] = "GC_CG"
                count += 1
            else: 
                count += 1
        yield sampleDict

# Test 
# for i in transitionsTransversions("/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/radio_new_vcf_2.xlsx", "/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/rradio_copy.fa", rradioOrigin):
#     print(i)

{'Sample_name': 'C10', 1763764: 'GC_TA', 3649628: 'GC_AT', 3649632: 'AT_TA', 3649633: 'AT_TA', 1763773: 'GC_AT', 1700377: 'AT_TA', 3468115: 'GC_AT', 3468112: 'AT_TA', 3468105: 'GC_AT'}
{'Sample_name': 'A6', 3414082: 'AT_CG', 916848: 'GC_TA', 1334795: 'AT_GC', 1802022: 'GC_AT'}
{'Sample_name': 'B3', 1763743: 'AT_TA', 2821955: 'GC_AT', 1763780: 'GC_TA', 1763773: 'GC_AT', 1763764: 'GC_TA', 1763757: 'AT_TA', 3649633: 'AT_GC', 3649632: 'AT_TA', 3649608: 'AT_GC', 3649604: 'AT_GC', 3649600: 'GC_TA', 3649598: 'AT_CG', 3649594: 'AT_GC'}
{'Sample_name': 'B11', 32834: 'GC_AT', 1855904: 'AT_GC'}
{'Sample_name': 'A4', 1445019: 'GC_TA'}
{'Sample_name': 'B6', 3547482: 'GC_AT'}
{'Sample_name': 'C9', 309078: 'GC_AT', 2761226: 'GC_AT', 953688: 'GC_AT', 664656: 'GC_CG', 3476734: 'GC_AT'}
{'Sample_name': 'D6', 3411906: 'AT_CG', 3038214: 'AT_TA', 375872: 'GC_TA', 1622319: 'GC_AT'}
{'Sample_name': 'D2', 779064: 'GC_TA'}
{'Sample_name': 'B2', 310768: 'GC_CG', 591827: 'AT_GC', 3765708: 'GC_CG'}
{'Sample_name'

In [27]:
#Categorize parts of the chromosome as early or late according to the possible scenarios 

#Use the generator object from the referenceStats function 
#Assumptions: the first and second chromosomes are the first and second generator objects, make sure this is true! 

#Set early and late mutation positions according to Scenario 1
#Early replicating is half the length of chromosome 2 divided in half around the origin of chromosome 1
#Both forks fire at the same time on both chromosomes 
#So length of chromosome 2 divided by 4. 
def scenario1(pathToReferenceFasta): 
    genObj = referenceStats(pathToReferenceFasta)
    
    for i in islice(genObj, 0, 1):
        chrom1Len = i[1]
    
    for i in islice(genObj, 0, 1):
        chrom2Len = i[1]

    halfLength = int(chrom2Len/4)
    afterOrigin = halfLength 
    beforeOrigin = chrom1Len - halfLength 
    
    #Early covers position 1 to afterOrigin, and beforeOrigin to the full length of the chromosome
    #Late is every position other than early 
    return(afterOrigin, beforeOrigin)

#Set up early and late according to Scenario 2
#Early replicating is before chromosome 2 begins replicating. 
#The late replicating section is half the length of chromosome 2 on either side of the terminus.
#Replication on the first chromosome starts first, then both chromosomes finish replicating at the same time 
def scenario2(pathToReferenceFasta):
    genObj = referenceStats(pathToReferenceFasta)
    
    for i in islice(genObj, 0, 1):
        chrom1Len = i[1]
    
    for i in islice(genObj, 0, 1):
        chrom2Len = i[1]
        
    halfLength = int(chrom2Len/4)
    terminus = int(chrom1Len/2)
    afterTerm = terminus + halfLength 
    beforeTerm = terminus - halfLength 
    
    #Early covers any position before late 
    #Late covers positions from beforeTerm to afterTerm and includes the terminus 
    return(beforeTerm, afterTerm)

#Test 
# print(scenario1("/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/Agrobacterium_tumefaciens_full_wplasmids.fasta"))

In [28]:
#Calculate number of total AT and CG sites according to early and late replication positions 
#Returns total AT and CG sites for early mutation positions and total AT and CG sites for late mutation positions 
#Parameter of function is either scenario 1 or scenario 2 
def earlyLateConditional(scenario, pathToReferenceFasta): 
    genObj = referenceStats(pathToReferenceFasta)
    for i in islice(genObj, 0, 1):
        chrom1Seq = i[0].seq
        chrom1Len = i[1]
        if scenario == "scenario1":
            afterOrigin, beforeOrigin = scenario1(pathToReferenceFasta)
            
            AT_sites_early = chrom1Seq[0:afterOrigin].count("A") + chrom1Seq[0:afterOrigin].count("T") + \
            chrom1Seq[beforeOrigin:chrom1Len].count("A") + chrom1Seq[beforeOrigin:chrom1Len].count("T")
            
            CG_sites_early = chrom1Seq[0:afterOrigin].count("C") + chrom1Seq[0:afterOrigin].count("G") + \
            chrom1Seq[beforeOrigin:chrom1Len].count("C") + chrom1Seq[beforeOrigin:chrom1Len].count("G")
            
            AT_sites_late = chrom1Seq[afterOrigin:beforeOrigin].count("A") + \
            chrom1Seq[afterOrigin:beforeOrigin].count("T") 
            
            CG_sites_late = chrom1Seq[afterOrigin:beforeOrigin].count("C") + \
            chrom1Seq[afterOrigin:beforeOrigin].count("G") 
            
            return(AT_sites_early, CG_sites_early, AT_sites_late, CG_sites_late)
        
        elif scenario == "scenario2":
            beforeTerm, afterTerm = scenario2(pathToReferenceFasta)
            
            AT_sites_early = chrom1Seq[0:beforeTerm].count("A") + chrom1Seq[0:beforeTerm].count("T") + \
            chrom1Seq[afterTerm:chrom1Len].count("A") + chrom1Seq[afterTerm:chrom1Len].count("T")
            
            AT_sites_late = chrom1Seq[beforeTerm:afterTerm].count("A") + chrom1Seq[beforeTerm:afterTerm].count("T")
            
            CG_sites_early = chrom1Seq[0:beforeTerm].count("C") + chrom1Seq[0:beforeTerm].count("G") + \
            chrom1Seq[afterTerm:chrom1Len].count("C") + chrom1Seq[afterTerm:chrom1Len].count("G")
            
            CG_sites_late = chrom1Seq[beforeTerm:afterTerm].count("C") + chrom1Seq[beforeTerm:afterTerm].count("G")
            
            return(AT_sites_early, CG_sites_early, AT_sites_late, CG_sites_late)

#Test     
#print(earlyLateConditional("scenario2", "/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/Agrobacterium_tumefaciens_full_wplasmids.fasta"))



In [29]:
#Categorize transitions and transversions from the VCF/excel file as early or late mutations as well as mutation type 
#Stores information in a dictionary with lists corresponding to early, late and the sample name 
#Specify scenario 1 or 2 to output the correct dictionary 

def categorizeMutations(pathToExcelFile, pathToReferenceFasta, origin, scenario):
    if scenario == "scenario1":  

        afterOrigin, beforeOrigin = scenario1(pathToReferenceFasta)

        for i in transitionsTransversions(pathToExcelFile, pathToReferenceFasta, origin):
            sampleDict = {"Early":[], "Late":[]}
            for key in i.keys():
                if key == "Sample_name":
                    sampleDict.update({"Sample_name":i[key]})
                elif key != "Sample_name":
                    if key >= 1 and key <= afterOrigin: 
                        sampleDict["Early"].append(i[key])
                    elif int(key) >= beforeOrigin:
                        sampleDict["Early"].append(i[key])
                    else: 
                        sampleDict["Late"].append(i[key])
                        
            yield sampleDict
        
    elif scenario == "scenario2":
        
        beforeTerm, afterTerm = scenario2(pathToReferenceFasta)
        
        for i in transitionsTransversions(pathToExcelFile, pathToReferenceFasta, origin):
            sampleDict = {"Early":[], "Late":[]}
            for key in i.keys():
                if key == "Sample_name":
                    sampleDict.update({"Sample_name":i[key]})
                elif key != "Sample_name":
                    if key >= beforeTerm and key <= afterTerm: 
                        sampleDict["Late"].append(i[key])
                    elif key < beforeTerm or key > afterTerm: 
                        sampleDict["Early"].append(i[key])
                        
            yield sampleDict
                               
#Test       
for i in categorizeMutations("/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/radio_new_vcf_2.xlsx", "/Users/jadetakakuwa/Desktop/SungLab/currentData/Radio/rradio_copy.fa", rradioOrigin, "scenario1"):
    print(i)

{'Early': ['GC_AT', 'AT_TA', 'AT_TA', 'GC_AT', 'AT_TA', 'GC_AT'], 'Late': ['GC_TA', 'GC_AT', 'AT_TA'], 'Sample_name': 'C10'}
{'Early': ['AT_CG'], 'Late': ['GC_TA', 'AT_GC', 'GC_AT'], 'Sample_name': 'A6'}
{'Early': ['AT_GC', 'AT_TA', 'AT_GC', 'AT_GC', 'GC_TA', 'AT_CG', 'AT_GC'], 'Late': ['AT_TA', 'GC_AT', 'GC_TA', 'GC_AT', 'GC_TA', 'AT_TA'], 'Sample_name': 'B3'}
{'Early': ['GC_AT'], 'Late': ['AT_GC'], 'Sample_name': 'B11'}
{'Early': [], 'Late': ['GC_TA'], 'Sample_name': 'A4'}
{'Early': ['GC_AT'], 'Late': [], 'Sample_name': 'B6'}
{'Early': ['GC_AT', 'GC_AT'], 'Late': ['GC_AT', 'GC_AT', 'GC_CG'], 'Sample_name': 'C9'}
{'Early': ['AT_CG', 'GC_TA'], 'Late': ['AT_TA', 'GC_AT'], 'Sample_name': 'D6'}
{'Early': [], 'Late': ['GC_TA'], 'Sample_name': 'D2'}
{'Early': ['GC_CG', 'AT_GC', 'GC_CG'], 'Late': [], 'Sample_name': 'B2'}
{'Early': ['AT_CG', 'GC_TA', 'GC_AT'], 'Late': ['GC_AT', 'GC_AT', 'GC_CG'], 'Sample_name': 'C5'}
{'Early': [], 'Late': ['GC_AT', 'GC_AT'], 'Sample_name': 'D7'}
{'Early': ['G

In [87]:
#Generation times for all organisms 
generationTimeRadio = 2492
generationTimeVitis = 2512 
generationTimeTum = 5819

#Input sample name, mutation type and whether the mutation is classified as early or late 
#Calculates the conditional mutation rate and adjusts according to generation time 
def calculateConditionalRates(sampleDict, pathToReferenceFasta, generationTime, scenario):
    
    earlyDict = {"AT_GC": 0, "GC_AT": 0, "AT_TA": 0, "GC_TA": 0, "AT_CG": 0, "GC_CG": 0, "Sample_name": ""}
    lateDict = {"AT_GC": 0, "GC_AT": 0, "AT_TA": 0, "GC_TA": 0, "AT_CG": 0, "GC_CG": 0, "Sample_name": ""}
    
    if scenario == "scenario1": 
        AT_sites_early, CG_sites_early, AT_sites_late, CG_sites_late = earlyLateConditional("scenario1", pathToReferenceFasta)
        for key in sampleDict.keys():
            if key == "Early":
                for mutation in sampleDict[key]:
                    if mutation == "AT_GC":
                        earlyDict["AT_GC"] += 1
                    elif mutation == "GC_AT":
                        earlyDict["GC_AT"] += 1
                    elif mutation == "AT_TA":
                        earlyDict["AT_TA"] += 1
                    elif mutation == "GC_TA":
                        earlyDict["GC_TA"] += 1
                    elif mutation == "AT_CG":
                        earlyDict["AT_CG"] += 1
                    elif mutation == "GC_CG": 
                        earlyDict["GC_CG"] += 1 
                    else: 
                        pass 
            elif key == "Late":
                for mutation in sampleDict[key]:
                    if mutation == "AT_GC":
                        lateDict["AT_GC"] += 1
                    elif mutation == "GC_AT":
                        lateDict["GC_AT"] += 1
                    elif mutation == "AT_TA":
                        lateDict["AT_TA"] += 1
                    elif mutation == "GC_TA":
                        lateDict["GC_TA"] += 1
                    elif mutation == "AT_CG":
                        lateDict["AT_CG"] += 1
                    elif mutation == "GC_CG": 
                        lateDict["GC_CG"] += 1 
                    else: 
                        pass 
            elif key == "Sample_name":
                earlyDict["Sample_name"] = sampleDict[key] 
                lateDict["Sample_name"] = sampleDict[key]

    
        for key in earlyDict.keys():
            if key == "AT_GC" or key == "AT_TA" or key == "AT_CG":
                mutRate = earlyDict[key]/AT_sites_early
                mutRate = mutRate/generationTime
                earlyDict[key] = mutRate
            elif key == "GC_AT" or key == "GC_TA" or key == "GC_CG":
                mutRate = earlyDict[key]/CG_sites_early
                mutRate = mutRate/generationTime
                earlyDict[key] = mutRate 

        for key in lateDict.keys():
            if key == "AT_GC" or key == "AT_TA" or key == "AT_CG":
                mutRate = lateDict[key]/AT_sites_late
                mutRate = mutRate/generationTime
                lateDict[key] = mutRate
            elif key == "GC_AT" or key == "GC_TA" or key == "GC_CG":
                mutRate = lateDict[key]/CG_sites_late
                mutRate = mutRate/generationTime
                lateDict[key] = mutRate 
                
        return earlyDict, lateDict
    
    elif scenario == "scenario2": 
        AT_sites_early, CG_sites_early, AT_sites_late, CG_sites_late = earlyLateConditional("scenario2", pathToReferenceFasta)
        for key in sampleDict.keys():
            if key == "Early":
                for mutation in sampleDict[key]:
                    if mutation == "AT_GC":
                        earlyDict["AT_GC"] += 1
                    elif mutation == "GC_AT":
                        earlyDict["GC_AT"] += 1
                    elif mutation == "AT_TA":
                        earlyDict["AT_TA"] += 1
                    elif mutation == "GC_TA":
                        earlyDict["GC_TA"] += 1
                    elif mutation == "AT_CG":
                        earlyDict["AT_CG"] += 1
                    elif mutation == "GC_CG": 
                        earlyDict["GC_CG"] += 1 
                    else: 
                        pass 
            elif key == "Late":
                for mutation in sampleDict[key]:
                    if mutation == "AT_GC":
                        lateDict["AT_GC"] += 1
                    elif mutation == "GC_AT":
                        lateDict["GC_AT"] += 1
                    elif mutation == "AT_TA":
                        lateDict["AT_TA"] += 1
                    elif mutation == "GC_TA":
                        lateDict["GC_TA"] += 1
                    elif mutation == "AT_CG":
                        lateDict["AT_CG"] += 1
                    elif mutation == "GC_CG": 
                        lateDict["GC_CG"] += 1
                    else: 
                        pass 
            elif key == "Sample_name":
                earlyDict["Sample_name"] = sampleDict[key] 
                lateDict["Sample_name"] = sampleDict[key]

    
        for key in earlyDict.keys():
            if key == "AT_GC" or key == "AT_TA" or key == "AT_CG":
                mutRate = earlyDict[key]/AT_sites_early
                mutRate = mutRate/generationTime
                earlyDict[key] = mutRate
            elif key == "GC_AT" or key == "GC_TA" or key == "GC_CG":
                mutRate = earlyDict[key]/CG_sites_early
                mutRate = mutRate/generationTime
                earlyDict[key] = mutRate 

        for key in lateDict.keys():
            if key == "AT_GC" or key == "AT_TA" or key == "AT_CG":
                mutRate = lateDict[key]/AT_sites_late
                mutRate = mutRate/generationTime
                lateDict[key] = mutRate
            elif key == "GC_AT" or key == "GC_TA" or key == "GC_CG":
                mutRate = lateDict[key]/CG_sites_late
                mutRate = mutRate/generationTime
                lateDict[key] = mutRate 

    return earlyDict, lateDict

#Create empty dataframes for early and late mutations 
final_mutations_early = pd.DataFrame(columns=["AT_GC", "GC_AT", "AT_TA", "GC_TA", "AT_CG", "GC_CG", "Sample_name"])
final_mutations_late = pd.DataFrame(columns=["AT_GC", "GC_AT", "AT_TA", "GC_TA", "AT_CG", "GC_CG", "Sample_name"])

#For each sample, create a dictionary for early and late mutations
#Add the dictionaries to the early and late mutation dataframes 
for i in categorizeMutations("/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/tum_new_vcf_2.xlsx", "/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/Agrobacterium_tumefaciens_full_wplasmids.fasta", tumOrigin, "scenario2"):
    earlyDict, lateDict = calculateConditionalRates(i, "/Users/jadetakakuwa/Desktop/SungLab/currentData/Tumefaciens/Agrobacterium_tumefaciens_full_wplasmids.fasta", generationTimeTum, "scenario2")
    final_mutations_early = final_mutations_early.append(earlyDict, ignore_index=True)
    final_mutations_late = final_mutations_late.append(lateDict, ignore_index=True)

#Write information to an excel file 
final_mutations_early.to_excel("tum_scenario2_early.xlsx")
final_mutations_late.to_excel("tum_scenario2_late.xlsx")